In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
from keras import layers
from keras.layers import Input, Dense, Dropout, Activation, BatchNormalization, Add
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPool1D, ZeroPadding1D, LSTM, Bidirectional, Flatten, GlobalAveragePooling2D
from keras.models import Sequential, Model
from keras.layers.merge import concatenate
from scipy.io import loadmat

In [2]:
SNOMED_scored=pd.read_csv("C://Abhijeet_Repo/PhysioNet-CinC-Challenge2020-TeamUIO-main/PhysioNet-CinC-Challenge2020-TeamUIO-main/Data/SNOMED_mappings_scored.csv", sep=";")
SNOMED_unscored=pd.read_csv("C://Abhijeet_Repo/PhysioNet-CinC-Challenge2020-TeamUIO-main/PhysioNet-CinC-Challenge2020-TeamUIO-main/Data/SNOMED_mappings_unscored.csv", sep=";")

In [3]:
snomed_class_names=["Pacing Rhythm", "Prolonged QT Interval","Atrial Fibrillation","Atrial Flutter",
                 "Left Bundle Branch Block","Q Wave Abnormal","T Wave Abnormal","Prolonged PR Interval","Ventricular Premature Beats",
"Low QRS Voltages","1st Degree AV Block","Premature Atrial Contraction","Left Axis Deviation",
"Sinus Bradycardia","Bradycardia","Sinus Rhythm","Sinus Tachycardia","Premature Ventricular Contractions",
"Sinus Arrhythmia","Left Anterior Fascicular Block","Right Axis Deviation","Right Bundle Branch Block","T Wave Inversion",
"Supraventricular Premature Beats","Nonspecific Intraventricular Conduction Disorder","Incomplete Right Bundle Branch Block",
"Complete Right Bundle Branch Block"]

In [141]:
header_file="C://Abhijeet_Repo/Data/A/raw/Training_WFDB/A0003.hea"
mat_file="C://Abhijeet_Repo/Data/A/raw/Training_WFDB/A0003.mat"

In [142]:
import pandas as pd
with open(header_file, 'r') as the_file:
    all_data = [line.strip() for line in the_file.readlines()]
    data = all_data[8:]
snomed_number=int(data[7][5:14])
value_unscored=SNOMED_unscored["Dx"][SNOMED_unscored["SNOMED CT Code"]==snomed_number].values
value_scored=SNOMED_scored["Dx"][SNOMED_scored["SNOMED CT Code"]==snomed_number].values
try: 
    disease_unscored=value_unscored[0]
except:
    disease_unscored=""

try: 
    disease_scored=value_scored[0]
except:
    disease_scored=""

In [143]:
lenet_5_model=Sequential()

lenet_5_model.add(Conv1D(filters=64, kernel_size=5, padding='same', input_shape=(5000,12)))
#lenet_5_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))
lenet_5_model.add(BatchNormalization())
lenet_5_model.add(Activation('relu'))

lenet_5_model.add(Conv1D(filters=64, kernel_size=3, padding='same',))
#lenet_5_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))
lenet_5_model.add(BatchNormalization())
lenet_5_model.add(Activation('relu'))

lenet_5_model.add(GlobalAveragePooling1D())

lenet_5_model.add(Dense(64, activation='relu'))

lenet_5_model.add(Dense(64, activation='relu'))

lenet_5_model.add(Dense(27, activation = 'softmax'))

In [144]:
lenet_5_model.load_weights("C://Users/ASudhakar/Downloads/lenet5_model_weights.best.hdf5")

In [145]:
yhat=lenet_5_model.predict(x=loadmat(mat_file)['val'].reshape(1,loadmat(mat_file)['val'].shape[1],loadmat(mat_file)['val'].shape[0]))
print("Predicted: "+snomed_class_names[np.argmax(yhat)])
if disease_unscored!="":
    print("Actual: "+disease_unscored)
else:
    print("Actual: "+disease_scored)

Predicted: Atrial Fibrillation
Actual: atrial fibrillation


In [146]:
lstm_model = Sequential()
lstm_model.add(LSTM(64, return_sequences=True, input_shape=(5000,12)))
lstm_model.add(LSTM(64, return_sequences=True))
lstm_model.add(LSTM(32, return_sequences=True))
lstm_model.add(GlobalAveragePooling1D())
lstm_model.add(Dense(32, activation = 'relu'))
lstm_model.add(Dense(27, activation = 'softmax'))

In [147]:
lstm_model.load_weights("C://Users/ASudhakar/Downloads/lstm_model_weights.best.hdf5")

In [148]:
yhat=lstm_model.predict(x=loadmat(mat_file)['val'].reshape(1,loadmat(mat_file)['val'].shape[1],loadmat(mat_file)['val'].shape[0]))
print("Predicted: "+snomed_class_names[np.argmax(yhat)])
if disease_unscored!="":
    print("Actual: "+disease_unscored)
else:
    print("Actual: "+disease_scored)

Predicted: Sinus Tachycardia
Actual: atrial fibrillation


In [149]:
def identity_block(X, f, filters):
    F1, F2, F3 = filters
    
    X_shortcut = X
    
    X = Conv1D(filters = F1, kernel_size = 1, activation='relu', strides = 1, padding = 'valid')(X)
    X = BatchNormalization()(X)
    
    X = Conv1D(filters = F2, kernel_size = f, activation='relu', strides = 1, padding = 'same')(X)
    X = BatchNormalization()(X)

    X = Conv1D(filters = F3, kernel_size = 1, activation='relu', strides = 1, padding = 'valid')(X)
    X = BatchNormalization()(X)

    X = Add()([X,X_shortcut])
    X = Activation('relu')(X)
    
    return X

def convolutional_block(X, f, filters, s = 2):
    F1, F2, F3 = filters
    
    X_shortcut = X

    X = Conv1D(F1, 1, activation='relu', strides = s)(X)
    X = BatchNormalization()(X)
    
    X = Conv1D(F2, f, activation='relu', strides = 1, padding = 'same')(X)
    X = BatchNormalization()(X)

    X = Conv1D(F3, 1, strides = 1, activation='relu')(X)
    X = BatchNormalization()(X)

    X_shortcut = Conv1D(F3, 1, strides = s)(X_shortcut)
    X_shortcut = BatchNormalization()(X_shortcut)
    
    X = Add()([X,X_shortcut])
    X = Activation('relu')(X)
    
    return X

def ResNet50(input_shape):
    
    X_input = Input(input_shape)

    X = ZeroPadding1D(3)(X_input)
    
    X = Conv1D(64, 7, strides = 2, activation='relu')(X)
    X = BatchNormalization()(X)
    X = MaxPool1D(pool_size=2, strides=2, padding='same')(X)

    X = convolutional_block(X, f = 3, filters = [64, 64, 256], s = 1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])

    X = convolutional_block(X, f = 3, filters = [128,128,512], s = 2)
    X = identity_block(X, 3, [128,128,512])
    X = identity_block(X, 3, [128,128,512])
    X = identity_block(X, 3, [128,128,512])

    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], s = 2)
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])

    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], s = 2)
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])

    X = MaxPool1D(pool_size=2, strides=2, padding='same')(X)
    
    X = GlobalAveragePooling1D()(X)
    X = Dense(27,activation='softmax')(X)
    
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [150]:
resNet50_model = ResNet50(input_shape = (5000,12))

In [151]:
resNet50_model.load_weights("C://Users/ASudhakar/Downloads/resnet50_model_weights.best.hdf5")

In [152]:
yhat=resNet50_model.predict(x=loadmat(mat_file)['val'].reshape(1,loadmat(mat_file)['val'].shape[1],loadmat(mat_file)['val'].shape[0]))
print("Predicted: "+snomed_class_names[np.argmax(yhat)])
if disease_unscored!="":
    print("Actual: "+disease_unscored)
else:
    print("Actual: "+disease_scored)

Predicted: Sinus Tachycardia
Actual: atrial fibrillation


In [153]:
vgg_16_model=Sequential()

vgg_16_model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu', input_shape=(5000,12)))
vgg_16_model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
vgg_16_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))
vgg_16_model.add(BatchNormalization())

vgg_16_model.add(Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))
vgg_16_model.add(Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))
vgg_16_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))
vgg_16_model.add(BatchNormalization())

vgg_16_model.add(Conv1D(filters=256, kernel_size=3, activation='relu', padding='same'))
vgg_16_model.add(Conv1D(filters=256, kernel_size=3, activation='relu', padding='same'))
vgg_16_model.add(Conv1D(filters=256, kernel_size=3, activation='relu', padding='same'))
vgg_16_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))
vgg_16_model.add(BatchNormalization())

vgg_16_model.add(Conv1D(filters=512, kernel_size=3, activation='relu', padding='same'))
vgg_16_model.add(Conv1D(filters=512, kernel_size=3, activation='relu', padding='same'))
vgg_16_model.add(Conv1D(filters=512, kernel_size=3, activation='relu', padding='same'))
vgg_16_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))
vgg_16_model.add(BatchNormalization())

vgg_16_model.add(Conv1D(filters=512, kernel_size=3, activation='relu', padding='same'))
vgg_16_model.add(Conv1D(filters=512, kernel_size=1, activation='relu', padding='same'))
vgg_16_model.add(Conv1D(filters=512, kernel_size=1, activation='relu', padding='same'))
vgg_16_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))
vgg_16_model.add(BatchNormalization())

vgg_16_model.add(GlobalAveragePooling1D())
vgg_16_model.add(Dense(256, activation='relu'))
vgg_16_model.add(Dropout(0.25))
vgg_16_model.add(Dense(128, activation='relu'))
vgg_16_model.add(Dropout(0.25))
vgg_16_model.add(Dense(27, activation='softmax'))

In [154]:
vgg_16_model.load_weights("C://Users/ASudhakar/Downloads/vgg16_model_weights.best.hdf5")

In [155]:
yhat=vgg_16_model.predict(x=loadmat(mat_file)['val'].reshape(1,loadmat(mat_file)['val'].shape[1],loadmat(mat_file)['val'].shape[0]))
print("Predicted: "+snomed_class_names[np.argmax(yhat)])
if disease_unscored!="":
    print("Actual: "+disease_unscored)
else:
    print("Actual: "+disease_scored)

Predicted: Atrial Fibrillation
Actual: atrial fibrillation
